In [26]:
import os
import pickle
import numpy as np
import physio
import matplotlib.pyplot as plt
import neurokit2 as nk
from customLib.preprocess import myConv1D, norm_min_max, invert_log_softmax

In [27]:
%matplotlib qt

In [103]:
PATH = "E:/ml-data/masters-thesis/myDataset/resp_prediction/mixed"
POST_PROCESS_PATH = "E:/ml-data/masters-thesis/myDataset/resp_prediction/postprocess/mixed"

aidmed_resp_sampling_rate = 50
aidmed_ecg_sampling_rate = 250

files = os.listdir(PATH)
print(len([x for x in files if "targets.npy" in x]))

2


In [104]:
def detect_local_extrema(x: np.array, y: np.array):
    """
    Detects local maxima and minima using the second derivative test.
    
    Args:
    - x: np.array, independent variable (e.g., time or position)
    - y: np.array, dependent variable (e.g., function values)
    
    Returns:
    - local_maxima: list of tuples (x, y) of local maxima points
    - local_minima: list of tuples (x, y) of local minima points
    """
    # Step 1: Calculate the first and second derivatives using numpy's gradient function
    dy_dx = np.gradient(y, x)   # First derivative
    d2y_dx2 = np.gradient(dy_dx, x)  # Second derivative

    # Step 2: Identify points where the first derivative crosses zero (potential extrema)
    local_maxima = []
    local_minima = []
    
    for i in range(1, len(dy_dx) - 1):
        if np.sign(dy_dx[i-1]) != np.sign(dy_dx[i+1]):  # Derivative changes sign
            if d2y_dx2[i] < 0:  # Negative second derivative => local maxima
                local_maxima.append((x[i]))
            elif d2y_dx2[i] > 0:  # Positive second derivative => local minima
                local_minima.append((x[i]))
    
    return np.array(local_maxima), np.array(local_minima)

def detect_error_indices(target, prediction, neighbourhood_size, window_length):
    # Create a binary array where target peak indices are set to 1
    target_prob = np.zeros(window_length)
    target_prob[target] = 1

    # Create a binary array where prediction peak indices are set to 1
    predict_prob = np.zeros(window_length)
    predict_prob[prediction] = 1

    # Convert prediction to a list for easier appending
    prediction = list(prediction)

    # Detect false positives
    for i, index in enumerate(prediction):
        print("Pred index: ", index)

        start_idx = int(max(0, index - neighbourhood_size))
        end_idx = int(min(index + neighbourhood_size, window_length - 1))

        target_slice = target_prob[start_idx:end_idx + 1]

        if not np.any(target_slice == 1):
            # Mark as false positive, ensure it appears twice as negative
            prediction[i] = -1 * index
            prediction.append(-1 * index)

    # Detect false negatives
    for i, index in enumerate(target):
        print("Target index: ", index)
        start_idx = int(max(0, index - neighbourhood_size))
        end_idx = int(min(index + neighbourhood_size, window_length - 1))

        prediction_slice = predict_prob[start_idx:end_idx + 1]

        if not np.any(prediction_slice == 1):
            # Mark as false negative (append only once)
            prediction.append(-1 * index)
        elif np.sum(prediction_slice) > 1:
            # More than one prediction in the neighborhood
            indexes = np.where(prediction_slice == 1)[0] + start_idx
            distances = np.abs(indexes - index)
            nearest_index = np.argmin(distances)

            # False positives are the non-nearest ones
            false_positives = np.delete(indexes, nearest_index)

            for fp in false_positives:
                i = prediction.index(fp)
                if i < len(prediction):
                    prediction[i] = -1 * fp
                prediction.append(-1 * fp)  # Append duplicate for false positive

        # Convert prediction back to numpy array
    return np.sort(np.array(prediction, dtype=int))


def calculate_fn_fp_indexes(prediction):
    unique, counts = np.unique(prediction[prediction < 0], return_counts=True)
    counts_dict = dict(zip(unique, counts))

    fn = np.abs(np.array([k for k, v in counts_dict.items() if v == 1]))
    fp = np.abs(np.array([k for k, v in counts_dict.items() if v == 2]))

    return fn, fp


def load_annotations(file_prefix, targets=True, dir="./"):
    choice = "targets"

    if not targets:
        choice = "prediction"

    try:
        resp = np.load(os.path.join(dir, str(file_prefix) + f"_{choice}.npy"))
        with open(os.path.join(dir, str(file_prefix) + f"_{choice}.peaks"), "rb") as f:
            peaks = pickle.load(f)
        with open(os.path.join(dir, str(file_prefix) + f"_{choice}.cycles"), "rb") as f:
            cycles = pickle.load(f)

    except Exception as e:
        print(e)
        return None, None, None

    return resp, peaks, cycles

def annotate_signal(file_prefix, dir, bpm="", targets=False):

    choice = "prediction"
    if targets:
        choice = "targets"

    file = str(file_prefix) + f"_{choice}.npy"

    # load signal
    resp = np.load(os.path.join(dir, file))

    resp = invert_log_softmax(resp)
    resp = norm_min_max(resp, -1, 1)
    resp = myConv1D(resp, kernel_length=100, padding="same")

    samples = [x for x in range(0, len(resp))]

    if bpm == "large":
        peaks, cycles = detect_local_extrema(samples, resp)
        peaks = peaks[::2]
        cycles = cycles[::2]

    elif bpm == "small":
        # When breaths per minute are small, there is a pause between the expiration of the first cycle and an inspiration of the follow up cycle
        # therefore physio is used to calculate the cycles indices as mean index of expiration and inspiration
        # and neurokit is used to predict the peaks of resp signal
        # other option is to calculate the mean index as the mean of expi + inspi index

        _, cycles = physio.compute_respiration(resp, aidmed_resp_sampling_rate, parameter_preset='human_airflow')
        inspi_index = cycles['inspi_index'].values
        expi_index = cycles['expi_index'].values
        cycles = np.sort(np.concatenate((inspi_index, expi_index)))

        peaks = nk.rsp_findpeaks(resp, sampling_rate=50)['RSP_Peaks']
    else:
        raise Exception("Wrong bpm parameter. Use either 'large' or 'small'")

    return resp, peaks, cycles


def load_postprocess(file_prefix, dir):
    target_resp, target_peaks, target_cycles = load_annotations(file_prefix, dir=dir)
    predicted_resp, predicted_peaks, predicted_cycles = load_annotations(file_prefix, targets=False, dir=dir)

    assert target_resp.shape == predicted_resp.shape

    return target_resp, predicted_resp, target_peaks, predicted_peaks, target_cycles, predicted_cycles

In [105]:
recording = 0

# annoptating the predictions
# target_resp, target_peaks, target_cycles = load_annotations(file_prefix=recording, dir=PATH, targets=True)
# predicted_resp, predicted_peaks, predicted_cycles = annotate_signal(file_prefix=recording, dir=PATH, targets=False, bpm="large")

# after annotating the files
target_resp, target_peaks, target_cycles = load_annotations(file_prefix=recording, dir=POST_PROCESS_PATH, targets=True)
predicted_resp, predicted_peaks, predicted_cycles = load_annotations(file_prefix=recording, dir=POST_PROCESS_PATH, targets=False)

inputs = np.load(os.path.join(PATH, str(recording) + "_inputs.npy"))
ecg, rrs, rpa = [inputs[:,:,x].flatten() for x in range(inputs.shape[2])]

###### load postprocessed targets and prediction after annotating it with manually + using first/second order derivative or neurokit
# target_resp, predicted_resp, target_peaks, predicted_peaks, target_cycles, predicted_cycles = load_postprocess(recording, dir=POST_PROCESS_PATH)

t = np.array([x * 1 / aidmed_resp_sampling_rate for x in range(len(target_resp))])
t_ecg = np.array([x * 1 / aidmed_ecg_sampling_rate for x in range(len(ecg))])
#t_ticks = np.concatenate((t[::aidmed_resp_sampling_rate * 20], [int(len(prediction_resp) * 1 / aidmed_resp_sampling_rate)]), axis=-1)

In [106]:
print(f"target_resp shape: {target_resp.shape}")
print(f"predicted_resp shape: {predicted_resp.shape}")
print(f"target_peaks shape: {target_peaks.shape}")
print(f"predicted_peaks shape: {predicted_peaks.shape}")
print(f"target_cycles shape: {target_cycles.shape}")
print(f"predicted_cycles shape: {predicted_cycles.shape}")
print(f"ECG shape: {ecg.shape}")
print(f"RRS shape: {rrs.shape}")
print(f"EPA shape: {rpa.shape}")

target_resp shape: (11750,)
predicted_resp shape: (11750,)
target_peaks shape: (33,)
predicted_peaks shape: (33,)
target_cycles shape: (34,)
predicted_cycles shape: (34,)
ECG shape: (58750,)
RRS shape: (58750,)
EPA shape: (58750,)


In [107]:
peaks_to_plot = np.copy(predicted_peaks)
cycles_to_plot = np.copy(predicted_cycles)

# running second time after fixing peaks and cycles
errors_peaks = np.where(predicted_peaks < 0)[0]
errors_cycles = np.where(predicted_cycles < 0)[0]

if len(errors_peaks) > 0:
    fn_peaks, fp_peaks = calculate_fn_fp_indexes(predicted_peaks)
    peaks_to_plot = peaks_to_plot[peaks_to_plot > 0]
else:
    fn_peaks, fp_peaks = [], []


if len(errors_cycles) > 0:
    fn_cycles, fp_cycles = calculate_fn_fp_indexes(predicted_cycles)
    cycles_to_plot = cycles_to_plot[cycles_to_plot > 0]
else:
    fn_cycles, fp_cycles = [], []


fig, axs = plt.subplots(nrows=3, sharex=True)

ax = axs[0]
ax.plot(t_ecg, ecg, color='royalblue')
ax.plot(t_ecg, rrs, color='red', linewidth=3)
ax.plot(t_ecg, rpa, color='green', linewidth=3)
ax.grid()
ax.legend(["ECG", "RR intervals", "RPA"])
ax.set_title("Inputs", fontweight="semibold")


ax = axs[1]
ax.plot(t, target_resp, 'k-')
ax.scatter(t[target_peaks], target_resp[target_peaks], s=50, color="orange", marker="o")
ax.scatter(t[target_cycles], target_resp[target_cycles], s=50, color="purple", marker="o")
ax.grid()
ax.set_title("Target", fontweight="semibold")
ax.legend(["Resp", "Peaks", "Cycles"])
#ax.set_xticks(t_ticks)

second_legend = []

ax = axs[2]
ax.plot(t, predicted_resp, 'dimgrey')
ax.scatter(t[peaks_to_plot], predicted_resp[peaks_to_plot], s=50, color="red", marker="o")
ax.scatter(t[cycles_to_plot], predicted_resp[cycles_to_plot], s=50, color="dodgerblue", marker="o")
second_legend += ["Resp", "Peaks", "Cycles"]
if len(fn_peaks) > 0:
    ax.scatter(t[fn_peaks], predicted_resp[fn_peaks], s=75, color="green", marker="X")
    second_legend += ["FN peaks"]
if len(fp_peaks) > 0:
    ax.scatter(t[fp_peaks], predicted_resp[fp_peaks], s=75, color="orange", marker="X")
    second_legend += ["FP peaks"]
if len(fn_cycles) > 0:
    ax.scatter(t[fn_cycles], predicted_resp[fn_cycles], s=75, color="darkorchid", marker="s")
    second_legend += ["FN cycles"]
if len(fp_cycles) > 0:
    ax.scatter(t[fp_cycles], predicted_resp[fp_cycles], s=75, color="violet", marker="s")
    second_legend += ["FP cycles"]
ax.grid()
ax.set_title("Predicted", fontweight="semibold")
ax.legend(second_legend)
#ax.set_xticks(t_ticks)

fig.supxlabel("Time [s]", fontweight="semibold")
fig.set_size_inches(8,8)

### Fixing peaks and cycles indices
Double negative value - FP
Single negative value - FN

In [45]:
target_peaks = np.concatenate((target_peaks, [265], [5205], [8600]))

In [54]:
predicted_peaks = predicted_peaks[predicted_peaks > 0]

predicted_peaks = detect_error_indices(target_peaks, predicted_peaks, 50, len(target_resp))

Pred index:  252
Pred index:  1062
Pred index:  1297
Pred index:  1604
Pred index:  1866
Pred index:  2173
Pred index:  2506
Pred index:  2886
Pred index:  3543
Pred index:  3990
Pred index:  4265
Pred index:  4632
Pred index:  5055
Pred index:  5352
Pred index:  5654
Pred index:  5885
Pred index:  6144
Pred index:  6426
Pred index:  6627
Pred index:  6910
Pred index:  7198
Pred index:  7845
Pred index:  8346
Pred index:  8721
Pred index:  8849
Pred index:  9125
Pred index:  9359
Pred index:  9574
Pred index:  9788
Pred index:  10017
Pred index:  10290
Pred index:  10571
Pred index:  10861
Pred index:  11135
Pred index:  11443
Pred index:  -1297
Pred index:  -2506
Pred index:  -8849
Target index:  244
Target index:  1037
Target index:  1618
Target index:  1885
Target index:  2154
Target index:  2421
Target index:  2881
Target index:  3553
Target index:  3974
Target index:  4273
Target index:  4618
Target index:  5029
Target index:  5346
Target index:  5634
Target index:  5882
Target in

In [55]:
predicted_peaks[predicted_peaks < 0]

array([-8910, -8849, -8849, -2506, -2506, -2421, -1297, -1297])

In [56]:
# predicted_cycles = predicted_cycles[predicted_cycles > 0]

predicted_cycles = detect_error_indices(target_cycles, predicted_cycles, 100, len(target_resp))

Pred index:  55
Pred index:  881
Pred index:  1251
Pred index:  1460
Pred index:  1741
Pred index:  2006
Pred index:  2288
Pred index:  2726
Pred index:  3372
Pred index:  3813
Pred index:  4120
Pred index:  4457
Pred index:  4880
Pred index:  5200
Pred index:  5500
Pred index:  5746
Pred index:  6024
Pred index:  6272
Pred index:  6532
Pred index:  6768
Pred index:  7019
Pred index:  7667
Pred index:  8171
Pred index:  8546
Pred index:  8798
Pred index:  8995
Pred index:  9251
Pred index:  9452
Pred index:  9687
Pred index:  9877
Pred index:  10130
Pred index:  10426
Pred index:  10724
Pred index:  10992
Pred index:  11275
Pred index:  11700
Pred index:  -881
Pred index:  -1251
Pred index:  -1460
Pred index:  -3372
Target index:  63
Target index:  682
Target index:  1359
Target index:  1768
Target index:  2018
Target index:  2291
Target index:  2638
Target index:  3236
Target index:  3777
Target index:  4132
Target index:  4440
Target index:  4836
Target index:  5189
Target index:  55

In [57]:
predicted_cycles[predicted_cycles < 0]

array([-3372, -3372, -3236, -1460, -1460, -1359, -1251, -1251,  -881,
        -881,  -682])

In [59]:
with open(POST_PROCESS_PATH + str(recording) + "_prediction.peaks", "wb") as f:
    pickle.dump(predicted_peaks, f)

with open(POST_PROCESS_PATH + str(recording) + "_prediction.cycles", "wb") as f:
    pickle.dump(predicted_cycles, f)


with open(POST_PROCESS_PATH + str(recording) + "_targets.peaks", "wb") as f:
    pickle.dump(target_peaks, f)

with open(POST_PROCESS_PATH + str(recording) + "_targets.cycles", "wb") as f:
    pickle.dump(target_cycles, f)

np.save(POST_PROCESS_PATH + str(recording) + "_targets.npy", target_resp)
np.save(POST_PROCESS_PATH + str(recording) + "_prediction.npy", predicted_resp)


### Calculating metrics using manual annotations

In [60]:
dir = "E:/ml-data/masters-thesis/myDataset/resp_prediction/postprocess/mixed/"

files = os.listdir(dir)

peaks_annotations = [x for x in files if "prediction.peaks" in x]
cycles_annotations = [x for x in files if "prediction.cycles" in x]

In [61]:
def ncc(x, y):
    """
    Computes the normalized cross-correlation coefficient for input signals.
    The input signal x can be of shape [batch_size, seq_length] or [seq_length].
    The input signal y can be of shape [batch_size, seq_length] or [seq_length].
    Returns a mean value of the cross-correlation coefficients.
    """

    # Reshape x and y if they are 1D (seq_length only) to [1, seq_length]
    if x.ndim == 1:
        x = np.expand_dims(x, 0)  # Shape becomes [1, seq_length]

    if y.ndim == 1:
        y = np.expand_dims(y, 0)  # Shape becomes [1, seq_length]

    # Ensure x and y have the same shape
    assert x.shape == y.shape, "X and Y must be of the same shape"
    
    # Compute the mean along the sequence dimension (dim=1)
    x_mean = np.mean(x, axis=1, keepdims=True)
    y_mean = np.mean(y, axis=1, keepdims=True)

    # Subtract the mean from the inputs
    x_prime = x - x_mean
    y_prime = y - y_mean

    # Compute the covariance
    covariance = np.sum(x_prime * y_prime, axis=1)

    # Compute the standard deviations
    std_x = np.sqrt(np.sum(x_prime ** 2, axis=1))
    std_y = np.sqrt(np.sum(y_prime ** 2, axis=1))

    # Compute the denominator and avoid division by zero
    denominator = std_x * std_y
    r_xy = np.where(denominator != 0, covariance / denominator, np.zeros_like(covariance))
    
    # Return the mean of the cross-correlation coefficients
    return np.mean(r_xy)


def filter_indices(array, filter_values):
    valid_indices = [i for i, val in enumerate(array) if val not in np.abs(filter_values)]
    return valid_indices


def calculate_cycles_data(targets, prediction):
    detected_cycles = 0
    fp_cycles = 0
    fn_cycles = 0

    detected_cycles_duration = 0.0
    detected_cycles_boundaries_time_deviation = 0.0

    # count TP, FP, FN for cycles prediction
    true_positive_cycles_indexes = prediction[prediction > 0]

    false_negative_cycles_indexes, false_positive_cycles_indexes = calculate_fn_fp_indexes(prediction)

    all_errors = np.sort(np.concatenate((false_negative_cycles_indexes, false_positive_cycles_indexes))).astype(int)
    all_indexes = np.sort(np.abs(prediction))

    if len(all_errors) > 0:
        valid_indices = filter_indices(all_indexes, all_errors)
        false_negative_indices = [i for i, v in enumerate(all_indexes) if v in false_negative_cycles_indexes]

        for i in range(1, len(valid_indices)):
            # if indexes are neighbours (checking their position in a list), then there is a cycle
            first_indice = valid_indices[i]
            second_indice = valid_indices[i-1]

            if first_indice - second_indice == 1:
                detected_cycles += 1
                detected_cycles_duration += abs(all_indexes[first_indice] - all_indexes[second_indice])

        if len(false_negative_indices) > 1:
            # count false negative cycles
            for i in range(1, len(false_negative_indices)):
                if false_negative_indices[i] - false_negative_indices[i-1] != 1:
                    fn_cycles += 2
                else:
                    fn_cycles += 1
        elif len(false_negative_indices) == 1:
            fn_cycles += 2
    else:
        detected_cycles += len(all_indexes) - 1
        detected_cycles_duration += np.sum(np.diff(all_indexes))
        
    # small respiration rate - false positive cycle influences the correct cycles
    # false positive peak indicates a false positive cycle
    # therefore instead of one point splitting two ground truth cycles, there are two, indicating start and end of FP cycle
    # error will be calculated as the two differences - abs(start of FP cycle - target) and (end - target)

    targets = targets[~np.isin(targets, false_negative_cycles_indexes)]

    fp_cycles += len(false_positive_cycles_indexes)

    detected_cycles_boundaries_time_deviation = np.sum(np.abs(np.sort(true_positive_cycles_indexes) - np.sort(targets)))

    return detected_cycles, fp_cycles, fn_cycles, detected_cycles_duration, detected_cycles_boundaries_time_deviation

In [63]:
peaks_time_deviation = 0.0
cycles_boundaries_time_deviation = 0.0

total_recording_length = 0.0

total_peaks = 0
detected_peaks = 0
fp_detected_peaks = 0
fn_detected_peaks = 0

total_cycles = 0
detected_cycles = 0
fp_detected_cycles = 0
fn_detected_cycles = 0

total_cycles_duration = 0.0
detected_cycles_duration = 0.0

total_ncc = 0.0
total_L1 = 0.0

print("File:", end=" ")
for i in range(len(peaks_annotations)):
    file_prefix = peaks_annotations[i].split("_")[0]
    print(file_prefix, end=" ")

    # load target resp and annotations
    target_resp, predicted_resp, target_peaks, predicted_peaks, target_cycles, predicted_cycles = load_postprocess(file_prefix, dir=dir) # for fast_breathing and normal_breathing

    total_peaks += len(target_peaks)                                            # total number of peaks
    total_cycles += len(target_cycles) - 1                                      # total number of cycles
    total_cycles_duration += np.sum(np.diff(np.sort(target_cycles)))            # total duration of cycles in samples
    total_recording_length += len(target_resp) / aidmed_resp_sampling_rate / 60 # total duration of recordings in minutes

    # count TP, FP, FN for peaks prediction
    true_positive_peaks = predicted_peaks[predicted_peaks > 0]
    unique, counts = np.unique(predicted_peaks[predicted_peaks < 0], return_counts=True)
    counts_dict = dict(zip(unique, counts))

    false_negative_peaks = np.abs(np.array([k for k, v in counts_dict.items() if v == 1]))
    fn_detected_peaks += len(false_negative_peaks)

    false_positive_peaks = np.abs(np.array([k for k, v in counts_dict.items() if v == 2]))
    fp_detected_peaks += len(false_positive_peaks)

    # Handle prediction errors (negative values)
    peaks_errors_indexes = predicted_peaks[predicted_peaks < 0]
    cycles_errors_indexes = predicted_cycles[predicted_cycles < 0]

    detected_peaks += len(np.unique(predicted_peaks))
    # remove error peaks indices from the predictions
    if len(peaks_errors_indexes) > 0:
        predicted_peaks = predicted_peaks[~np.isin(predicted_peaks, peaks_errors_indexes)]
        target_peaks = target_peaks[~np.isin(target_peaks, np.abs(peaks_errors_indexes))]
        
    # Calculate time deviation between predicted and target peaks
    peaks_time_deviation += np.sum(np.abs(np.sort(target_peaks) - np.sort(predicted_peaks)))

    dc, fp, fn, dcd, dcbtd = calculate_cycles_data(target_cycles, predicted_cycles)

    detected_cycles += dc
    fp_detected_cycles += fp
    fn_detected_cycles += fn
    detected_cycles_duration += dcd
    cycles_boundaries_time_deviation += dcbtd

    total_ncc += ncc(target_resp, predicted_resp)
    total_L1 += np.sum(np.abs(target_resp - predicted_resp))

avg_peak_time_deviation = peaks_time_deviation / detected_peaks / aidmed_resp_sampling_rate if detected_peaks > 0 else 0
avg_peak_detection_rate = detected_peaks / total_peaks if total_peaks > 0 else 0

avg_cycle_time_deviation = cycles_boundaries_time_deviation / detected_cycles / aidmed_resp_sampling_rate if detected_cycles > 0 else 0
avg_cycle_detection_rate = (detected_cycles + fp_detected_cycles) / total_cycles if detected_cycles > 0 else 0

avg_detected_cycles_duration = detected_cycles_duration / detected_cycles / aidmed_resp_sampling_rate if detected_cycles > 0 else 0
avg_total_cycles_duration = total_cycles_duration / total_cycles / aidmed_resp_sampling_rate if total_cycles > 0 else 0

print(f"\nAverage peak time deviation: {avg_peak_time_deviation:>11.3f} [s] | Peak detection rate: {avg_peak_detection_rate:>16.4f}\n"
      f"Average cycle time deviation: {avg_cycle_time_deviation:>10.3f} [s] | Avg cycle detection rate: {avg_cycle_detection_rate:>11.4f}\n"
      f"Average detected cycles duration: {avg_detected_cycles_duration:>6.3f} [s] | Average total cycles duration: {avg_total_cycles_duration:>5.3f} [s]\n"
      f"\nTotal recordings duration: {total_recording_length:.2f} [min] \nTotal peaks: {total_peaks} \nAvg breaths per minute: {total_peaks / total_recording_length:.2f}\n"
      f"\nFP peaks count: {fp_detected_peaks} \nFN peaks count: {fn_detected_peaks}\n"
      f"\nTotal cycles: {total_cycles} \nFP cycles count: {fp_detected_cycles} \nFN cycles count: {fn_detected_cycles}\n"
      f"\nAvg NCC: {total_ncc / len(peaks_annotations):.4f}\n"
      f"Avg L1: {total_L1 / len(peaks_annotations):.4f}\n"
      )

File: 0 1 
Average peak time deviation:       0.297 [s] | Peak detection rate:           1.0448
Average cycle time deviation:      0.435 [s] | Avg cycle detection rate:      0.9851
Average detected cycles duration:  6.583 [s] | Average total cycles duration: 6.903 [s]

Total recordings duration: 7.83 [min] 
Total peaks: 67 
Avg breaths per minute: 8.55

FP peaks count: 3 
FN peaks count: 2

Total cycles: 67 
FP cycles count: 4 
FN cycles count: 4

Avg NCC: 0.8436
Avg L1: 2608.4028

